In [1]:
import time
import logging

import krpc
import numpy as np
from IPython.display import clear_output

from UPG import UPG, Status
import utils
import Apollo


In [2]:
conn = krpc.connect()
space_center = conn.space_center
vessel = space_center.active_vessel
ap = vessel.auto_pilot
body = vessel.orbit.body
body_frame = body.reference_frame
flight = vessel.flight(body_frame)

# body constants
r_e = body.equatorial_radius
g_0 = body.surface_gravity

In [3]:
lon = -60.0
lat = 0.0
# The touch down velocity.
final_velocity = 3.
# The target height above target when guidance ends.
final_height = 50. 
# The final acceleration when guidance ends.
# Some tests shows that large values will use more fuel, but small values will lead to a deep trajectory.
final_acc = 1.2
# Minimum throttle. This must be exactly equal to your engine's min throttle.
min_throttle = 0.1  
# First burn time, in secondes.
t_1 = 10.  
# Engine's rated burn time, or the time you want to burn at most.
rated_burn_time = 960 
# k_r for aapdg algorithm. Should fall in range [6,12], and larger value 
# means more fuel but more gentle trajectory. Personnaly I recommend 8
k_r = 8.0 
# if your throttle saturate, and thrust error is always negative, 
# then your time is too short. Increase this value to lower thrust.
time_coef = 1.2

# if you find your craft turning too slow, alter the following values.
# The time for craft to stop rotating. Since the torque is constant, the 
# large these values is, the fast you will rotate.
st = 0.9
ap.stopping_time = (st, st, st)
# ap.deceleration_time = (3., 3., 3.)
# This controls the precision. Smaller values will lead to unstablity.
aa = 0.6
ap.attenuation_angle = (aa, aa, aa)

In [4]:
# target vector, with final height added.
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height + final_height, body_frame))

# target reference frame.
target_frame = utils.target_reference_frame(space_center, body, lat, lon)
# get final velocity and up vector in body frame
up = r_target / np.linalg.norm(r_target)
v_target = -final_velocity * up

# draw the target
line = conn.drawing.add_line((0., 0., 0.), (20., 0., 0.), target_frame)
line.thickness = 2

In [5]:
specific_impulse = conn.add_stream(getattr, vessel,
                                          'specific_impulse')
max_thrust = conn.add_stream(getattr, vessel, 'available_thrust')
current_thrust = conn.add_stream(getattr, vessel, 'thrust')
mass = conn.add_stream(getattr, vessel, 'mass')
situation = conn.add_stream(getattr, vessel, 'situation')
ut = conn.add_stream(getattr, space_center, 'ut')
height = conn.add_stream(getattr, flight, 'surface_altitude')
mean_altitude = conn.add_stream(getattr, flight, 'mean_altitude')
speed = conn.add_stream(getattr, flight, 'speed')
vertical_speed = conn.add_stream(getattr, flight, 'vertical_speed')
velocity = conn.add_stream(vessel.velocity, body_frame)
position = conn.add_stream(vessel.position, body_frame)

In [6]:
# get initial values
v_0 = utils.vector(velocity())
r_0 = utils.vector(position())
# initial guess
pv = -v_0 / np.linalg.norm(v_0)
pr = np.zeros((3, 1))
t_f = np.array([[rated_burn_time]]) / 2

print("Initiating UPG")
upg = UPG(r_0=r_e, g_0=g_0, r_target=r_target, v_target=v_target, r_t=r_0, v_t=v_0,
    mass=mass(), max_thrust=max_thrust(), min_throttle=min_throttle, 
    specific_impulse=specific_impulse(), k=0., t_0=ut(), t_1=t_1, p_r_0=pr, p_v_0=pv,
    t_f=t_f, t_2_bound=[5, rated_burn_time / 20.])

print("Evaluating t2...")
upg.solve_t_2()


Initiating UPG
Evaluating t2...
Terminate program if it stucks at solving t_2.


In [7]:
# PDI determination
print("Waiting for powered descent initial")
ap.reference_frame = body_frame
ap.engage()
guidance_interval = 0.1

apdg = Apollo.APDG(r_target, final_velocity, final_acc, t_f, ut(), up, mass(), r_0, v_0, g_0, k_r)

while True:
    v_0 = utils.vector(velocity())
    r_0 = utils.vector(position())
    
    if upg.status == Status.PDI:
        upg.update_state(r_t=r_0, v_t=v_0, mass=mass(),
               max_thrust=max_thrust(), t_0=ut())
        upg.update_start_time(ut())
        upg.solve(mode='soft')
        r_guidance = utils.downrange(r_0, upg.r_final, body, body_frame)
        r_togo = utils.downrange(r_0, r_target, body, body_frame)
        print("guidance downrange: {0:,.2f}m, target downrange: {1:,.2f}m"
            .format(r_guidance, r_togo))
        
        
        if r_guidance > r_togo - 3e4:
            t_f = time_coef * upg.t_f + ut()
            apdg.t_f = t_f
            apdg.update(r_0, v_0, mass(), ut())
            apdg.compute()
            ap.target_direction = apdg.thrust_direction
        else:
            ap.target_direction = upg.thrust_direction
        # start descent when overshoot the target with margin of 1km.
        if r_guidance > r_togo + 1000.:
            print("descent start!")
            upg.status = Status.Powered_descent
            m_initial = mass()
            continue

    elif upg.status == Status.Powered_descent:
        apdg.update(r_0, v_0, mass(), ut())
        apdg.compute()

        vessel.control.throttle = utils.thrust2throttle(
        apdg.thrust, max_thrust(), min_throttle)
        ap.target_direction = apdg.thrust_direction

        print("t_go: {:.2f} s".format(apdg.t_go), end=' ')
        r_go = np.linalg.norm(r_target - r_0)
        print("distance to land: {:,.2f} m".format(r_go))
        print("thrust error: {:.2f} kN".format((vessel.thrust - apdg.thrust) / 1e3))
        print("AP direction error {:.2f}".format(ap.error))

        clear_output(wait=True)
        if apdg.t_go < 5 or r_go < 10 or height() < final_height:
            print("AAPDG disengaged")
            break
    clear_output(wait=True)
    time.sleep(guidance_interval)


while True:
    v_0 = utils.vector(velocity())
    r_0 = utils.vector(position())
    acc_dir, acc = Apollo.gravity_turn(vertical_speed(), speed(), mean_altitude(), final_velocity, target_height, g_0, r_0, v_0)
    ap.target_direction = acc_dir
    vessel.control.throttle = utils.thrust2throttle(
        mass() * acc, max_thrust(), min_throttle)
    if situation() == space_center.VesselSituation.landed \
            or situation() == space_center.VesselSituation.splashed\
            or vertical_speed() > -0.5:
        print("landed!")
        break
ap.disengage()
vessel.control.throttle = 0.

r_0 = utils.vector(position())
target_height = body.surface_height(lat, lon)
r_target = utils.vector(body.position_at_altitude(
    lat, lon, target_height, body_frame))
print("Landing precision: {:.2f} m".format(utils.downrange(r_0, r_target, body, body_frame)))
print("Delta-v used: {:.2f} m/s".format(specific_impulse() *  9.80665 * np.log(m_initial / mass())))


AAPDG disengaged
landed!
Landing precision: 2.54 m
Delta-v used: 1839.55 m/s
